<a href="https://colab.research.google.com/github/HabibaAbderrahim/advanced_data-preparation-tool-pack/blob/main/gsheet_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import os
import gspread
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Main directory containing plant subfolders on Google Drive
main_directory = '/content/drive/MyDrive/Folder_Analysis'

In [ ]:
# Authenticate with Google Sheets API
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Create an empty DataFrame to store all data
combined_data = pd.DataFrame()

In [ ]:
# Loop through each subfolder
for plant_folder in os.listdir(main_directory):
    plant_path = os.path.join(main_directory, plant_folder)

    # Check if it's a directory
    if os.path.isdir(plant_path):
        for file in os.listdir(plant_path):
            if file.endswith('.gsheet'):  # Handle Google Sheets
                file_name = os.path.splitext(file)[0]  # Remove the file extension
                plant_name = plant_folder  # Assign the folder name as "Plant Name"

                # Attempt to open the Google Sheets document by title with retries
                for _ in range(3):  # Try 3 times
                    try:
                        worksheet = gc.open(file_name).sheet1  # Use file_name as the title
                        data = worksheet.get_all_records()
                        df = pd.DataFrame(data)

                        # Add a "Plant Name" column and set it to the current plant's name
                        df['Plant Name'] = plant_name
                        combined_data = combined_data.append(df, ignore_index=True)
                        break  # Success, break out of the retry loop
                    except gspread.SpreadsheetNotFound:
                        print(f"Google Sheets document with title '{file_name}' not found.")
                        break  # No need to retry if the document is not found
                    except gspread.exceptions.APIError as e:
                        print(f"APIError: {e}")
                        print("Retrying in 5 seconds...")
                        time.sleep(5)  # Wait for 5 seconds before retrying

# Save the combined data to a CSV file
combined_data.to_csv('/content/drive/My Drive/combined_plant_data_gsheet.csv', index=False)
